# Plan observable transits tonight: 

In [1]:
%matplotlib inline
from astroplan import (FixedTarget, Observer, is_event_observable, 
                       AltitudeConstraint, AtNightConstraint, 
                       MoonSeparationConstraint)
from astroplan.periodic import EclipsingSystem
import astropy.units as u
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table


>>> from astroplan import download_IERS_A
>>> download_IERS_A()
 [astroplan.utils]


Download a table of known transiting exoplanets: 

Create an astroplan [`EclipsingSystem`](https://astroplan.readthedocs.io/en/latest/api/astroplan.EclipsingSystem.html) object for each transiting exoplanet: 

In [3]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.table import Column

nasa_exoplanet_table = NasaExoplanetArchive.get_confirmed_planets_table(cache=False, all_columns=True)
nasa_transit_table = nasa_exoplanet_table[nasa_exoplanet_table['pl_discmethod'] == 'Transit']
nasa_transit_table.add_index('pl_name')

In [273]:
from astropy.constants import G, M_sun, R_sun

inc = nasa_transit_table['pl_orbincl']
inc[inc == 0*u.deg] = 90*u.deg
# a = nasa_transit_table['pl_orbsmax']
period = nasa_transit_table['pl_orbper']

# Kepler's law
mstar = nasa_transit_table['st_mass']
mstar[mstar == 0*M_sun] = 1 * M_sun
a = (( G*mstar / (4*np.pi**2) * period**2 )**(1/3) ).decompose()

rp = nasa_transit_table['pl_radj']
rstar = nasa_transit_table['st_rad']
rstar[rstar == 0*R_sun] = 1*R_sun
ecc = nasa_transit_table['pl_orbeccen']
arstar = (a/rstar).decompose()
arstar[arstar == 0] = 10
b = arstar * np.cos(inc)
#rprstar = nasa_transit_table['pl_ratror']
rprstar = (rp/rstar).decompose()
ecc = nasa_transit_table['pl_orbeccen'].filled(0)
omega = np.radians(nasa_transit_table['pl_orblper'].filled(90))

durations = period/np.pi * np.arcsin( np.sqrt((1 + rprstar)**2 - b**2) / np.sin(inc) / arstar).value * np.sqrt(1 - ecc**2) / (1 + ecc*np.sin(omega))
durations[np.isnan(durations)] = 0*u.day

/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/units/quantity.py:639: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [274]:
from astropy.constants import R_jup, R_sun
epochs = nasa_transit_table['pl_tranmid']
names = nasa_transit_table['pl_name']
simbad_names = nasa_transit_table['pl_name']
radius_ratio = (nasa_transit_table['pl_radj'] / nasa_transit_table['st_rad']).decompose().value
sky_coords = nasa_transit_table['sky_coord']

eclipsing_systems = []
transit_depths = []
coords = []
for p, t0, dur, name, rprs, coord in zip(periods, epochs, durations, names, radius_ratio, sky_coords):
    if not any([isinstance(i, np.ma.core.MaskedConstant) for i in [p, t0, dur]]):
        eclisping_sys = EclipsingSystem(Time(t0, format='jd'), p, duration=dur, name=name)
        eclipsing_systems.append(eclisping_sys)
        transit_depths.append(rprs**2)
        coords.append(coord)

Initialize the observatory, and say we're observing tonight: 

In [282]:
obs = Observer.at_site('KPNO')
start, end = obs.tonight(Time('2018-12-28'), horizon=-12*u.deg)

Only compute observablility for transits of exoplanets with $R_p/R_\star >$ `minimum_radius_ratio` :

In [288]:
minimum_depth = 0 # 0.0011
n_days = 1
altitude_limit = 15 * u.deg

transit_events = []
labels = 'Name, Depth, Ingress, Egress, Alt1, Alt2'.split(', ')

for day in range(n_days):
    transits_day = []
    for es, coord, depths in zip(eclipsing_systems, coords, transit_depths):

        next_transit = es.next_primary_ingress_egress_time(start, n_eclipses=1)
        ingress = next_transit[0, 0]
        egress = next_transit[0, 1]

        if egress < end+day*u.day and ingress > start+day*u.day: 
            ingress_alt = obs.altaz(ingress, coord).alt
            egress_alt = obs.altaz(egress, coord).alt
            observable = (ingress_alt > altitude_limit) and (egress_alt > altitude_limit)
            if observable: 
                transits_day.append([es.name, round(depths, 4), next_transit[0, 0].iso, next_transit[0, 1].iso, 
                                     round(ingress_alt.degree, 0),
                                     round(egress_alt.degree, 0)])
    if len(transits_day) > 0:
        transit_events.append(Table(rows=transits_day, names=labels))
    else: 
        transit_events.append(None)

In [289]:
for i, table in enumerate(transit_events):
    print('\n\nTransits on {0} UTC:'.format((start+i*u.day).datetime.date()))
    if table is not None: 
        table.sort('Depth')
        table.pprint(max_width=1000, max_lines=1000)
    else:
        print('None')



Transits on 2018-12-28 UTC:
      Name       Depth          Ingress                  Egress         Alt1 Alt2
---------------- ------ ----------------------- ----------------------- ---- ----
        K2-187 b 0.0002 2018-12-28 05:14:35.553 2018-12-28 06:49:32.735 29.0 49.0
        K2-257 b 0.0002 2018-12-28 10:42:48.632 2018-12-28 12:03:29.144 37.0 50.0
        K2-111 b 0.0002 2018-12-28 02:44:10.855 2018-12-28 05:52:04.729 58.0 74.0
EPIC 220674823 b 0.0003 2018-12-28 03:32:47.551 2018-12-28 05:05:43.608 59.0 41.0
       HD 3167 b 0.0003 2018-12-28 03:14:55.535 2018-12-28 04:52:51.675 54.0 36.0
         K2-35 c 0.0004 2018-12-28 10:09:47.188 2018-12-28 11:51:54.335 46.0 59.0
        K2-183 b 0.0006 2018-12-28 08:03:18.506 2018-12-28 08:03:18.506 65.0 65.0
        K2-275 b 0.0008 2018-12-28 05:16:31.553 2018-12-28 07:28:05.957 29.0 56.0
        K2-151 b 0.0009 2018-12-28 03:24:04.804 2018-12-28 05:05:48.818 63.0 45.0
        K2-117 b 0.0013 2018-12-28 08:17:12.662 2018-12-28 09:34:25.